In [1]:
!pip install stanza
import stanza.pipeline
stanza.download('en')
nlp_stanza = stanza.Pipeline('en',processors= 'tokenize,ner')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 7.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| ner       | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


Load models

In [ ]:
#!python -m spacy download en_core_web_sm
#nlp = spacy.load("en_core_web_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Named Entity Recognition

In [3]:
import pandas as pd


In [9]:

def extract_entities_stanza(text):
    doc = nlp_stanza(text)
    entList = []
    for sentence in doc.sentences:
        for ent in sentence.ents:
            entList.append({
                "text": ent.text,
                "n_word": len(ent.text.split(" ")),
                "start_pos": ent.start_char,
                "end_pos": ent.end_char,
                "label": ent.type
            })
    return pd.DataFrame(entList)

df = pd.read_csv('/content/physics_and_chemistry_nobel_laureate.csv')

df_allents = pd.DataFrame()
for index, row in df.iterrows():
  text = row['biography']
  category = row['category']
  df_ents = extract_entities_stanza(text)
  df_ents['source_index'] = index
  df_ents["category"] = category

  df_allents = pd.concat([df_allents, df_ents], ignore_index = True)

print(df_allents)

                                                 text  n_word  start_pos  \
0                              Wilhelm Conrad Röntgen       3          0   
1                                              German       1         26   
2                                       27 March 1845       3         70   
3                                    10 February 1923       3         86   
4                                              German       1        110   
...                                               ...     ...        ...   
59847  the Wayback Machine\nBoard of Science Advisors       6       3646   
59848         The Buffalo International Film Festival       5       3693   
59849                                         Buffalo       1       3734   
59850                                              NY       1       3743   
59851                                   United States       2       3747   

       end_pos   label  source_index   category  
0           22  PERSON             0 

In [7]:
def computeStats(df_ents):

    stats_count = df_ents['label'].value_counts().describe()

    df_ents['length'] = df_ents['text'].apply(len)
    stats_length = df_ents.groupby('label')['length'].describe()

    return stats_count, stats_length



In [11]:
df_allents.to_csv("df_entities_stanza.csv", index=False)